In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from ipywidgets import interact, Dropdown
from IPython.display import display
import warnings

warnings.filterwarnings("ignore", message="An unsupported index was provided.*")
warnings.filterwarnings("ignore", message="No supported index is available.*")
pd.options.display.float_format = '{:.2f}'.format

In [7]:
df_climate = pd.read_csv('data/climate.csv')
df_climate['year'] = df_climate['year'].astype(int)
df_climate = df_climate.sort_values(['country_code', 'year'])
print(f'Loaded {len(df_climate)} rows for {df_climate['country_code'].nunique()} countries.')

Loaded 29363 rows for 180 countries.


In [14]:
def plot_climate_hw(country_code):
    subset = df_climate[df_climate['country_code'] == country_code].copy()
    if subset.empty:
        raise ValueError('No data for %s' % country_code)
    actual = subset[(subset['year'] >= 2003) & (subset['year'] <= 2020)].groupby('year')['temp_mean'].mean()
    train = actual[(actual.index >= 2003) & (actual.index <= 2017)]
    if len(train) < 10:
        raise ValueError('Not enough data for %s' % country_code)
    model = ExponentialSmoothing(train, trend='add', seasonal=None)
    fitted = model.fit(optimized=True)
    forecast_years = np.arange(2020, 2026)
    forecast = fitted.forecast(steps=6)
    forecast_values = forecast.values if hasattr(forecast, 'values') else np.asarray(forecast)
    if 2020 in actual.index:
        forecast_values = forecast_values.copy()
        forecast_values[0] = actual.loc[2020]
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(actual.index, actual.values, label='Actual (2003-2020)', color='tab:blue')
    last_year = actual.index[-1]
    last_value = actual.iloc[-1]
    ax.scatter([last_year], [last_value], color='tab:blue', s=100, edgecolor='white', label=f'Last actual ({last_year})')
    ax.plot(forecast_years, forecast_values, 'o--', color='tab:red', label='Forecast (2020-2025)')
    ax.set_title(f'Holt-Winters temp_mean forecast for {country_code}')
    ax.set_xlabel('Year')
    ax.set_ylabel('temp_mean')
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.show()
    forecast_df = pd.DataFrame({'year': forecast_years, 'forecast_temp_mean': forecast.round(2)})
    display(forecast_df)

In [15]:
interact(plot_climate_hw, country_code=Dropdown(options=sorted(df_climate['country_code'].unique()), value='USA', description='Country'))

interactive(children=(Dropdown(description='Country', index=170, options=('AFG', 'AGO', 'ALB', 'ARE', 'ARG', '…

<function __main__.plot_climate_hw(country_code)>